# Описание данных

Проект состоит из двух частей:

    В первой части вы решите несколько задач в SQL-тренажёре, чтобы закрепить пройденный материал.
    Вторая часть проекта — аналитическая. Проверять задачи по-прежнему будет тренажёр. Однако мы рекомендуем самостоятельно анализировать полученные результаты и формулировать выводы.
    
Внимательно читайте условия. Формулировки задач в проекте приближены к реальной жизни. В работе часто встретятся такие формулировки: «рассчитать LTV за последние 30 дней», «отобразить самых активных покупателей» или даже «определить, почему в компании упали продажи».

В самостоятельном проекте вы будете работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.

Вы будете работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах вы найдёте информацию и о более поздних оценках, которые эти посты получили. 

### Изучите ER-диаграмму базы:

![ER](https://pictures.s3.yandex.net/resources/Frame_353_1_1664969703.png)

### Первая часть

**1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».**

In [ ]:
SELECT COUNT(type)
FROM stackoverflow.posts
JOIN stackoverflow.post_types ON  stackoverflow.post_types.id = stackoverflow.posts.post_type_id 
WHERE type = 'Question' AND (score > 300 OR favorites_count >= 100)

**2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.**

In [ ]:
SELECT ROUND(AVG(q.count_), 0)
FROM (
      SELECT COUNT(id) AS count_,
             creation_date::date
      FROM stackoverflow.posts
      WHERE post_type_id = 1
      GROUP BY creation_date::date
      HAVING creation_date::date BETWEEN '2008-11-01' AND '2008-11-18') AS q;

**3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.**

In [ ]:
SELECT COUNT(DISTINCT b.user_id)
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS u ON u.id=b.user_id
WHERE u.creation_date::date = b.creation_date::date

**4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?**

In [ ]:
SELECT COUNT(tabl_1.id)
FROM (SELECT po.id
      FROM stackoverflow.posts AS po
      JOIN stackoverflow.votes AS v ON po.id=v.post_id
      JOIN stackoverflow.users AS u ON po.user_id=u.id
      WHERE u.display_name LIKE 'Joel Coehoorn' AND v.id >= 1
      GROUP BY po.id) AS tabl_1;

**5. Выгрузите все поля таблицы `vote_types`. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю `id`.**

In [ ]:
SELECT *,
    ROW_NUMBER() OVER (ORDER BY vote_types.id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY vote_types.id 

**6. Отберите 10 пользователей, которые поставили больше всего голосов типа `Close`.** 

**Отобразите таблицу из двух полей:** 
* **идентификатором пользователя**
* **количеством голосов.**

**Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.**

In [ ]:
SELECT *
FROM (SELECT v.user_id,
             COUNT(vt.id) AS v_count
      FROM stackoverflow.votes AS v
      JOIN stackoverflow.vote_types AS vt ON vt.id=v.vote_type_id
      WHERE vt.name LIKE 'Close'
      GROUP BY v.user_id
      ORDER BY v_count DESC
      LIMIT 10) AS tabl_1
ORDER BY tabl_1.v_count DESC, tabl_1.user_id DESC;

**7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.**

**Отобразите несколько полей:**
* **идентификатор пользователя;**
* **число значков;**
* **место в рейтинге — чем больше значков, тем выше рейтинг.**

**Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.**
**Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.**

In [ ]:
SELECT *,
      DENSE_RANK() OVER (ORDER BY table_1.b_count DESC) AS rating
FROM (SELECT user_id,
             COUNT(id) AS b_count
      FROM stackoverflow.badges
      WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15' 
      GROUP BY user_id
      ORDER BY b_count DESC, user_id LIMIT 10) as table_1;

**8. Сколько в среднем очков получает пост каждого пользователя?**

**Сформируйте таблицу из следующих полей:**
* **заголовок поста;**
* **идентификатор пользователя;**
* **число очков поста;**
* **среднее число очков пользователя за пост, округлённое до целого числа.**

**Не учитывайте посты без заголовка, а также те, что набрали ноль очков.**

In [ ]:
WITH table_1 AS (
            SELECT ROUND(AVG(score)) AS avg_score,
                   user_id
            FROM stackoverflow.posts
            WHERE title IS NOT NULL AND score != 0
            GROUP BY user_id
)

SELECT po.title,
       table_1.user_id,
       po.score,
       table_1.avg_score
FROM table_1
JOIN stackoverflow.posts AS po ON table_1.user_id=po.user_id
WHERE po.title IS NOT NULL AND po.score != 0;

**9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.**

In [ ]:
SELECT title
FROM stackoverflow.posts
WHERE user_id IN (SELECT user_id
                  FROM stackoverflow.badges
                  GROUP BY user_id
                  HAVING COUNT(id) >1000)
              AND title IS NOT NULL;

**10. Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:**
* **пользователям с числом просмотров больше либо равным 350 присвойте группу 1;**
* **пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;**
* **пользователям с числом просмотров меньше 100 — группу 3.**

**Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.**

In [ ]:
SELECT id,
       views,
       CASE
          WHEN views >= 350 THEN 1
          WHEN views < 350 and views >= 100 THEN 2
          WHEN views < 100 THEN 3
       END AS group
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views != 0;

**11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.**

In [ ]:
WITH table_1 AS (SELECT ca.id,
                    ca.views,
                    ca.group,
                    MAX(ca.views) OVER (PARTITION BY ca.group) AS max_   
                 FROM (SELECT 
                           id,
                           views,
                           CASE
                               WHEN views >= 350 THEN 1
                               WHEN views < 350 and views >= 100 THEN 2
                               WHEN views < 100 THEN 3
                           END AS group
                       FROM stackoverflow.users
                       WHERE location LIKE '%Canada%' AND views != 0) AS ca
)

SELECT table_1.id, 
       table_1.views,  
       table_1.group
FROM table_1
WHERE table_1.views = table_1.max_
ORDER BY table_1.views DESC, table_1.id;

**12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:**
* **номер дня;**
* **число пользователей, зарегистрированных в этот день;**
* **сумму пользователей с накоплением.**

In [ ]:
SELECT *,
       SUM(uc.count_reg) OVER (ORDER BY uc.days) as count_users
FROM (SELECT EXTRACT(DAY FROM creation_date::date) AS days,
             COUNT(id) AS count_reg
      FROM stackoverflow.users
      WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
      GROUP BY EXTRACT(DAY FROM creation_date::date)
      ) as uc;

**13. Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста.**

**Отобразите:**
* **идентификатор пользователя;**
* **разницу во времени между регистрацией и первым постом.**

In [ ]:
WITH table_1 AS (
                SELECT 
                    DISTINCT user_id,
                    MIN(creation_date) OVER (PARTITION BY user_id) AS min_dt      
                FROM stackoverflow.posts
)

SELECT 
    table_1.user_id,
    (table_1.min_dt - u.creation_date) AS diff
FROM stackoverflow.users AS u 
JOIN table_1 ON u.id = table_1.user_id;

### Вторая часть

**1. Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.**

In [ ]:
SELECT 
    DATE_TRUNC('month', creation_date)::date AS month_,
    SUM(views_count) 
FROM stackoverflow.posts
GROUP BY DATE_TRUNC('month', creation_date)
ORDER BY SUM(views_count) DESC;

**2. Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений `user_id`. Отсортируйте результат по полю с именами в лексикографическом порядке.**

In [ ]:
SELECT 
    u.display_name,
    COUNT(DISTINCT po.user_id)
FROM stackoverflow.posts AS po
JOIN stackoverflow.users AS u ON po.user_id=u.id
JOIN stackoverflow.post_types AS pt ON pt.id=po.post_type_id
WHERE po.creation_date::date BETWEEN u.creation_date::date AND (u.creation_date::date + INTERVAL '1 month') 
    AND pt.type LIKE '%Answer%'
GROUP BY u.display_name
HAVING COUNT(po.id) > 100
ORDER BY u.display_name;

**3. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.**

In [ ]:
WITH table_1 AS (
               SELECT 
                   u.id
               FROM stackoverflow.posts AS po
               JOIN stackoverflow.users AS u ON po.user_id=u.id
               WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01' 
                     AND DATE_TRUNC('month', po.creation_date)::date = '2008-12-01'
               GROUP BY u.id
               HAVING COUNT(po.id) != 0
)

SELECT 
    COUNT(po.id),
    DATE_TRUNC('month', po.creation_date)::date
FROM stackoverflow.posts AS po
WHERE po.user_id IN (SELECT * FROM table_1)
      AND DATE_TRUNC('year', po.creation_date)::date = '2008-01-01'
GROUP BY DATE_TRUNC('month', po.creation_date)::date
ORDER BY DATE_TRUNC('month', po.creation_date)::date DESC;

**4. Используя данные о постах, выведите несколько полей:**
* **идентификатор пользователя, который написал пост;**
* **дата создания поста;**
* **количество просмотров у текущего поста;**
* **сумма просмотров постов автора с накоплением.**

**Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.**

In [ ]:
SELECT 
    user_id,
    creation_date,
    views_count,
    SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts 
ORDER BY user_id;

**5. Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.**

In [ ]:
WITH table_1 AS (
                SELECT 
                    user_id,
                    COUNT(DISTINCT creation_date::date) AS count_
                FROM stackoverflow.posts
                WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07' 
                GROUP BY user_id
)

SELECT ROUND(AVG(table_1.count_))
FROM table_1;

**6. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:**
* **Номер месяца.**
* **Количество постов за месяц.**
* **Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.**

**Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип `numeric`.**

In [ ]:
WITH table_1 AS (SELECT 
                     EXTRACT(MONTH FROM creation_date::date) AS month_,
                     COUNT(DISTINCT id) AS count_   
                 FROM stackoverflow.posts
                 WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
                 GROUP BY month_)

SELECT *,
       ROUND(((count_::numeric / LAG(count_) OVER (ORDER BY month_)) - 1) * 100, 2) AS user_growth
FROM table_1;

**7. Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации.**

**Выведите данные его активности за октябрь 2008 года в таком виде:**
* **номер недели;**
* **дата и время последнего поста, опубликованного на этой неделе.**

In [ ]:
WITH table_1 AS (SELECT 
                     user_id,
                     COUNT(DISTINCT id) AS cnt
                 FROM stackoverflow.posts
                 GROUP BY user_id
                 ORDER BY cnt DESC
                 LIMIT 1
),

      table_2 AS (SELECT 
                      po.user_id,
                      po.creation_date,
                      EXTRACT('week' FROM po.creation_date) AS week_number
                  FROM stackoverflow.posts AS po
                  JOIN table_1 ON table_1.user_id = po.user_id
                  WHERE DATE_TRUNC('month', po.creation_date)::date = '2008-10-01')

SELECT DISTINCT 
    week_number::numeric,
    MAX(creation_date) OVER (PARTITION BY week_number) AS post_dt
FROM table_2
ORDER BY week_number;